In [ ]:
import numpy as np
from matplotlib import pyplot
import matplotlib
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
try:
    import path_guiding
except ModuleNotFoundError:
    import imp
    path_guiding = imp.load_dynamic('path_guiding', r'..\buildwin\Release\path_guiding.dll')
from path_guiding import GMM2d

In [ ]:
#import path_guiding_kdtree_loader
#import path_guiding_kdtree_visualization

In [ ]:
def pdf_image(ax, gmm):
    t = np.linspace(-1.,1., 100)
    x,y = np.meshgrid(t,t)
    coord_list = np.vstack((x.ravel(), y.ravel())).T
    pdf_vals = gmm.pdf(coord_list)
    pdf_vals = np.reshape(pdf_vals, (t.size,t.size))
    return ax.imshow(pdf_vals[::-1,:], extent=(-1.,1,-1.,1.))

def rot(a):
    a = np.pi/180*a
    cs = np.cos(a)
    sn = np.sin(a)
    rot = [
        [cs, sn],
        [-sn,cs]
    ]
    return np.matrix(rot)

In [ ]:
gmm = GMM2d.make_for_unit_circle()
pdf_image(pyplot.gca(), gmm)
pyplot.show()

Check if sampling and evaluation of pdf works
==============================================

In [ ]:
init_precisions = np.zeros((8,2,2), dtype=np.float32)

for k in range(8):
    a = np.random.uniform(0., 360.)
    m = np.matrix([
        [50., 0.],
        [0. , 500.]
    ])
    r = rot(a)
    init_precisions[k,...] = r * m * r.T

gmm = GMM2d.make_for_unit_circle()
gmm.precisions = init_precisions
w = (np.arange(8)+1).astype(np.float32)
gmm.weights = w / np.sum(w)
    
xs = gmm.sample(1000)

pdf_image(pyplot.gca(), gmm)
pyplot.scatter(*xs.T, color = 'gray', s = 5., marker='x')
pyplot.scatter(*gmm.means.T, color = 'r')

Try the fitting procedures
--------------------------

In [ ]:
gmm = GMM2d.make_for_unit_circle()

In [ ]:
def test_set1():
    N = 100
    loc1 = (-0.5,0.25)
    loc2 = (0.5,-0.25)
    test_data_two_gaussian = np.random.normal(loc=loc1, scale=0.33, size=(N,2))
    test_data_two_gaussian = np.vstack((test_data_two_gaussian, np.random.normal(loc=loc2, scale=0.33, size=(N,2))))
    weights = np.ones(N*2)
    return np.array([loc1,loc2]), test_data_two_gaussian, weights

def test_set2():
    N = 100
    loc1 = (-0.33,0.1)
    loc2 = (0.33,-0.1)
    x1 = np.random.normal(loc=0., scale=0.25, size=(N,2))
    x2 = np.random.normal(loc=0., scale=0.25, size=(N,2))
    x1[:,0] *= 0.2
    x2[:,0] *= 0.2
    x1 = np.dot(rot(30.), x1.T).T
    x2 = np.dot(rot(-30.), x2.T).T
    x1 += np.array(loc1)[np.newaxis,:]
    x2 += np.array(loc2)[np.newaxis,:]
    weights = np.ones(N*2)
    return np.array([loc1,loc2]), np.array(np.concatenate((x1,x2),axis=0)), weights

def test_set3():
    locs, xs, ws = test_set2()
    more_xs = np.random.uniform(-1., 1., size=(200,2))
    weights = np.ones(more_xs.shape[0])
    xs = np.vstack([xs, more_xs])
    ws = np.hstack([ws, weights])
    return locs, xs, ws

def test_set4():
    locs, xs, ws = test_set2()
    more_xs = np.random.uniform(-1., 1., size=(200,2))
    weights = 0.01*np.ones(more_xs.shape[0])
    xs = np.vstack([xs, more_xs])
    ws = np.hstack([ws, weights])
    return locs, xs, ws

def test_set5():
    locs, xs, ws = test_set2()
    bigx = [0.,1.]
    locs = np.vstack((locs, bigx))
    xs = np.vstack((xs, bigx))
    ws = np.hstack((ws, [100.]))
    return locs, xs, ws

def test_set6():
    xs = np.random.uniform(-1., 1., size = (50*100, 2))
    loc1 = (-0.33,0.1)
    loc2 = (0.33,-0.1)
    locs = np.array([loc1,loc2])
    ws = np.zeros(xs.shape[0])
    for l in locs:
        scale = 0.25
        ws += np.exp(-np.linalg.norm(xs - l[np.newaxis,:], axis=1)**2/scale/scale)
    return locs, xs, ws

def test_set7():
    locs, xs, ws = test_set1()
    ws[0:10] = 0
    ws[np.random.binomial(1, p=0.1, size = ws.shape[0]).astype(np.bool)] = 0
    return locs, xs, ws

In [ ]:
locs, xs, ws = test_set6()
pyplot.scatter(*xs.T, s = 40, c = ws)
print (xs.shape[0])

In [ ]:
prior_nu = 1.00001; prior_alpha = 2.01; prior_u = 1.e-5; max_iters = 1; maximization_step_every = 100;
#prior_nu = 1.01; prior_alpha = 2.01; prior_u = 1.e-5; max_iters = 50; maximization_step_every = 10;  # from paper

In [ ]:
gmm = GMM2d.make_for_unit_circle()
gmm.fit(xs[:400,...], ws[:400,...], prior_nu, prior_alpha, prior_u, max_iters)
fig, ax = pyplot.subplots(1,1, figsize=(10,20))
pdf_image(ax, gmm)
ax.scatter(*xs[:400,...].T, marker = 'x', s = 40, c = ws[:400])
ax.scatter(*locs.T, marker = '+', s = 40, color = 'r')
ax.scatter(*gmm.means.T, marker='o', s = 40, color='r')
pyplot.show()

Incremental Fit
===============

In [ ]:
prior_nu = 10.
prior_alpha = 10.
prior_tau = 1.
maximization_step_every = 100;
prior_mode = GMM2d.make_for_unit_circle()
prior_mode.precisions = prior_mode.precisions * np.random.uniform(1., 10., size=(8,))[:,np.newaxis,np.newaxis]
w = (np.arange(8)+1).astype(np.float32)
prior_mode.weights = w/np.sum(w)

gmm = GMM2d.make_for_unit_circle()
gmm.means = np.zeros_like(gmm.means)

incremental = path_guiding.GMM2dFitIncremental(
    prior_nu = prior_nu, 
    prior_alpha = prior_alpha, 
    prior_tau = prior_tau,
    prior_mode = prior_mode,
    maximization_step_every = maximization_step_every)

#permutation = np.arange(xs.shape[0]); np.random.shuffle(permutation)
pxs = xs
pws = ws

for i in range(0,pxs.shape[0],maximization_step_every):
    seen_xs = pxs[i:i+maximization_step_every,...]
    seen_ws = pws[i:i+maximization_step_every]
    incremental.fit(gmm, seen_xs, seen_ws)
    fig, ax = pyplot.subplots(1,1, figsize=(10,20))
    pdf_image(ax, gmm)
    ax.scatter(*seen_xs.T, marker = 'x', s = 40, color = 'gray')
    ax.scatter(*locs.T, marker = '+', s = 40, color = 'r')
    ax.scatter(*gmm.means.T, marker='o', s = 40, color='r')
    pyplot.show()

# incremental.fit(gmm, xs[permutation,:], ws[permutation])
# fig, ax = pyplot.subplots(1,1, figsize=(10,20))
# pdf_image(ax, gmm)
# ax.scatter(*xs.T, marker = 'x', s = 40, color = 'gray')
# ax.scatter(*locs.T, marker = '+', s = 40, color = 'r')
# ax.scatter(*gmm.means().T, marker='o', s = 40, color='r')
# pyplot.show()

Find nice initialization for centers of the gaussians in the mixture
-----------------------------------------------------------------------------------
Distribute uniformly in the unit circle. That is because I'll fit data in the unit circle only.

In [ ]:
# Bascially, compute repulsive forces on each point. 
# Points are mutually repulsive. And they are also repelled by the boundary of the circle.

def forces(pts):
    N = pts.shape[0]
    meanR = np.sqrt(1. / N)
    forces = np.zeros_like(pts)
    for i, p in enumerate(pts):
        for j, q in enumerate(pts):
            if i != j:
                d = q-p # points away from p
                l = np.linalg.norm(d)+1.e-9
                # repulsive?!
                f = -np.exp(-l/meanR)/l*d
                forces[i] += f
        # force of border
        l = np.linalg.norm(p)
        f = -np.exp((l-1.)/meanR)/l*p
        forces[i] += f
    return forces

# Integrate forces. Neglect momentum. Thus displacement is proportional to force.
pts = np.random.normal(loc=0., scale=0.1, size=(8,2)).astype(np.float32)
dt = 0.1
for iter in range(500):
    pts += dt*forces(pts)

In [ ]:
meanR = np.sqrt(1. / pts.shape[0])
pyplot.gca().add_patch(matplotlib.patches.Circle((0.,0.),radius=1, facecolor='none', edgecolor='r'))
for i in range(pts.shape[0]):
    pyplot.gca().add_patch(matplotlib.patches.Circle(pts[i],radius=meanR, facecolor='none', edgecolor='k'))
pyplot.scatter(*pts.T, marker = 'x', s = 40, color = 'k')
pyplot.gca().autoscale_view()

Now the same in 3d
-----------------------------

In [ ]:
def forces(pts):
    N = pts.shape[0]
    meanR = np.sqrt(1. / N)
    forces = np.zeros_like(pts)
    for i, p in enumerate(pts):
        for j, q in enumerate(pts):
            if i != j:
                d = q-p # points away from p
                l = np.linalg.norm(d)+1.e-9
                # repulsive?!
                f = -np.exp(-l/meanR)/l*d
                forces[i] += f
    return forces

def renormalize(pts):
    pts /= np.linalg.norm(pts, axis=1)[:,np.newaxis]

# Integrate forces. Neglect momentum. Thus displacement is proportional to force.
pts = np.random.normal(loc=0., scale=0.1, size=(8,3)).astype(np.float32)
renormalize(pts)
dt = 0.1
for iter in range(500):
    pts += dt*forces(pts)
    renormalize(pts)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
%matplotlib notebook

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*pts.T)

In [ ]:
x = np.float32(1.e-38)
np.float32(1.)/x

In [ ]:
for p in pts:
    print ("{{{}}},".format(','.join(map(str,p))))